In [9]:
import time
import qrcode
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import binascii
from pyzbar.pyzbar import decode
from PIL import Image

# Helper function to display data in hexadecimal
def hex_display(data):
    return binascii.hexlify(data).decode('utf-8')

# Helper function to generate and save QR code, then display it
def generate_qr_code(data, title):
    qr = qrcode.make(data)
    qr.save(f"{title}.png")  # Save the QR code image temporarily
    qr.show(title=title)
    return f"{title}.png"  # Return the filename

# Helper function to read QR code from saved image file
def read_qr_code(filename):
    with Image.open(filename) as img:
        decoded_data = decode(img)[0].data.decode("utf-8")
    return decoded_data

# Set up plaintext for encryption
plaintext = b'This is a test msg for AES!'  # Example plaintext
print("Plaintext:", hex_display(plaintext))

# Generate 192-bit and 256-bit keys
key1_192 = get_random_bytes(24)  # First 192-bit key
key2_256 = get_random_bytes(32)  # Second 256-bit key
print("Key 1 (192-bit):", hex_display(key1_192))
print("Key 2 (256-bit):", hex_display(key2_256))

# Initialization vector (IV) for AES CBC mode
iv = get_random_bytes(16)
print("Initialization Vector (IV):", hex_display(iv))

# Convert plaintext to QR Code, save and display it, then read back the text
plaintext_qr_filename = generate_qr_code(plaintext.decode("utf-8"), "Plaintext QR Code")
plaintext_from_qr = read_qr_code(plaintext_qr_filename)
print("Text from Plaintext QR Code:", plaintext_from_qr)

# Encrypt and Decrypt function for Double AES with execution time and error rate
def double_aes_encrypt_decrypt(plaintext, key1, key2, iv):
    # Start time for the entire process (encryption + decryption)
    start_time_total = time.perf_counter()

    # First layer encryption with key1 (AES-192)
    cipher1 = AES.new(key1, AES.MODE_CBC, iv)
    first_layer_ciphertext = cipher1.encrypt(plaintext.ljust(32))  # Pad to 32 bytes for AES
    print("First Layer Ciphertext:", hex_display(first_layer_ciphertext))

    # Second layer encryption with key2 (AES-256)
    cipher2 = AES.new(key2, AES.MODE_CBC, iv)
    final_ciphertext = cipher2.encrypt(first_layer_ciphertext)
    print("Final Ciphertext after Double AES Encryption:", hex_display(final_ciphertext))

    # Convert final ciphertext to QR Code, save and display it, then read back the ciphertext
    final_ciphertext_qr_filename = generate_qr_code(hex_display(final_ciphertext), "Ciphertext QR Code 192-256")
    ciphertext_from_qr = read_qr_code(final_ciphertext_qr_filename)
    print("Ciphertext from QR Code (Hex):", ciphertext_from_qr)

    # Convert ciphertext hex back to bytes for decryption
    final_ciphertext_from_qr = binascii.unhexlify(ciphertext_from_qr)

    # First layer decryption with key2
    decipher1 = AES.new(key2, AES.MODE_CBC, iv)
    first_layer_decrypted = decipher1.decrypt(final_ciphertext_from_qr)
    print("First Layer Decrypted Text:", hex_display(first_layer_decrypted))

    # Second layer decryption with key1
    decipher2 = AES.new(key1, AES.MODE_CBC, iv)
    final_decrypted_text = decipher2.decrypt(first_layer_decrypted).strip()
    print("Final Decrypted Text after Double AES Decryption:", final_decrypted_text.decode("utf-8"))

    # Convert decrypted text to QR Code, save and display it
    decrypted_text_qr_filename = generate_qr_code(final_decrypted_text.decode("utf-8"), "Decrypted Text QR Code 192-256")

    # End time for the entire process (encryption + decryption)
    end_time_total = time.perf_counter()
    total_execution_time = end_time_total - start_time_total
    print("Total Execution Time (Encryption + Decryption):", total_execution_time, "seconds")

    # Calculate Error Rate
    error_rate = sum(1 for a, b in zip(plaintext, final_decrypted_text) if a != b) / len(plaintext) * 100
    print("Error Rate:", error_rate, "%")

    return total_execution_time, error_rate

# Execute Double AES with a 192-bit key for the first layer and a 256-bit key for the second layer
print("\n--- Double AES with 192-bit and 256-bit Keys ---")
total_execution_time, error_rate = double_aes_encrypt_decrypt(plaintext, key1_192, key2_256, iv)


Plaintext: 5468697320697320612074657374206d736720666f722041455321
Key 1 (192-bit): 1d966ef56d2c34154f20947448ca1ef4d35f3f12bdd30380
Key 2 (256-bit): e1f8bc32f9ac1422c84665319fac1b49a72f6002e94907037aeae4e2e40fcb38
Initialization Vector (IV): 617bc4c238e7a184d1203a358dd2869a
Text from Plaintext QR Code: This is a test msg for AES!

--- Double AES with 192-bit and 256-bit Keys ---
First Layer Ciphertext: 242888326ea0cad2222439537f33732b28df0bb595d377bfe9a5961bc2fdb7ee
Final Ciphertext after Double AES Encryption: d1e4e8f59cfedd08aac4979f8ddf4fabacee3c80cfe7771d2e6ab8308ac09eb5
Ciphertext from QR Code (Hex): d1e4e8f59cfedd08aac4979f8ddf4fabacee3c80cfe7771d2e6ab8308ac09eb5
First Layer Decrypted Text: 242888326ea0cad2222439537f33732b28df0bb595d377bfe9a5961bc2fdb7ee
Final Decrypted Text after Double AES Decryption: This is a test msg for AES!
Total Execution Time (Encryption + Decryption): 7.233341200015275 seconds
Error Rate: 0.0 %
